In [ ]:
import sqlite3
import asyncio
import nest_asyncio
!playwright install-deps
!playwright install chromium
!pip install playwright langchain-openai nest_asyncio
from playwright.async_api import async_playwright

In [ ]:
def setup_db():
    conn = sqlite3.connect('negotiator_brain.db')
    cursor = conn.cursor()

    cursor.execute('''CREATE TABLE IF NOT EXISTS chats
                      (phone TEXT PRIMARY KEY, history TEXT, last_price INTEGER)''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS fraud_registry
                      (phone TEXT PRIMARY KEY, risk_score INTEGER, notes TEXT)''')
    conn.commit()
    conn.close()
    print("✅ Database Initialized.")

if __name__ == "__main__":
    setup_db()

✅ Database Initialized.


In [ ]:
class SafetyAgent:
    def __init__(self):
        self.red_flags = ["advance", "token", "jazzcash", "easypaisa", "delivery", "cargo"]

    def calculate_risk(self, message, price, market_avg=120000):
        score = 0

        if price < (market_avg * 0.6):
            score += 5

        for word in self.red_flags:
            if word in message.lower():
                score += 4
        return score


if __name__ == "__main__":
    scanner = SafetyAgent()
    msg = "Bhai bike mil jaye gi, pehlay 2000 advance bhenjo token ke liye."
    risk = scanner.calculate_risk(msg, 60000)
    print(f"🚩 Risk Score: {risk}/10 (High risk if > 6)")

🚩 Risk Score: 13/10 (High risk if > 6)


In [ ]:
nest_asyncio.apply()

class ColabOLXScraper:
    def __init__(self):
        self.base_url = "https://www.olx.com.pk/items/q-"

    async def get_ads(self, search_query):
        async with async_playwright() as p:

            browser = await p.chromium.launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])

            context = await browser.new_context(
                user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
            )

            page = await context.new_page()

            search_url = f"{self.base_url}{search_query.replace(' ', '-')}"
            print(f"📡 Requesting: {search_url}")


            await page.goto(search_url, wait_until="networkidle", timeout=60000)

            try:

                await page.wait_for_selector('[data-aut-id="itemBox"]', timeout=30000)

                ads = []
                items = await page.locator('[data-aut-id="itemBox"]').all()

                for item in items[:5]:
                    try:
                        title = await item.locator('[data-aut-id="itemTitle"]').inner_text()
                        price_str = await item.locator('[data-aut-id="itemPrice"]').inner_text()

                        price = int(''.join(filter(str.isdigit, price_str)))

                        ads.append({"title": title, "price": price})
                    except:
                        continue

                await browser.close()
                return ads
            except Exception as e:

                await page.screenshot(path="debug_error.png")
                print("❌ Timeout or Blocked. Saved 'debug_error.png' to see what went wrong.")
                await browser.close()
                return []


scraper = ColabOLXScraper()

search_results = asyncio.run(scraper.get_ads("honda 125 lahore"))

if search_results:
    print("\n✅ Successfully Found Ads:")
    for ad in search_results:
        print(f"🏍️ {ad['title']} | Price: PKR {ad['price']}")
else:
    print("\n⚠️ No listings found. Check 'debug_error.png' in Colab files.")

📡 Requesting: https://www.olx.com.pk/items/q-honda-125-lahore
❌ Timeout or Blocked. Saved 'debug_error.png' to see what went wrong.

⚠️ No listings found. Check 'debug_error.png' in Colab files.


In [ ]:
def run_negotiator(seller_num, seller_msg, price):

    safety = SafetyAgent()
    risk = safety.calculate_risk(seller_msg, price)

    if risk > 6:
        return "🛑 Potential Fraud Detected! Stopping negotiation."

    print("⚠️ HaggleBrain is not defined. Skipping AI response generation.")
    reply = "AI response functionality is temporarily disabled."

    return f"🤖 AI Response to {seller_num}: {reply}"

if __name__ == "__main__":
    setup_db()
    result = run_negotiator("+923211234567", "135,000 se kam nahi hoga.", 135000)
    print(result)

✅ Database Initialized.
⚠️ HaggleBrain is not defined. Skipping AI response generation.
🤖 AI Response to +923211234567: AI response functionality is temporarily disabled.


In [ ]:
async def scrape_top_ad(search_query):
    async with async_playwright() as p:

        browser = await p.chromium.launch(
            headless=True,
            args=['--no-sandbox', '--disable-blink-features=AutomationControlled']
        )


        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
            viewport={'width': 1280, 'height': 800}
        )

        page = await context.new_page()


        search_url = f"https://www.olx.com.pk/items/q-{search_query.replace(' ', '-')}"
        print(f"📡 Requesting: {search_url}")

        try:

            response = await page.goto(search_url, wait_until="load", timeout=60000)


            await asyncio.sleep(3)


            if "distil" in await page.content() or "captcha" in await page.content():
                print("🛑 OLX Security Wall detected. Saving screenshot to check...")
                await page.screenshot(path="blocked.png")
                await browser.close()
                return None


            await page.wait_for_selector('[data-aut-id="itemBox"]', timeout=15000)


            first_item = page.locator('[data-aut-id="itemBox"]').first
            title = await first_item.locator('[data-aut-id="itemTitle"]').inner_text()
            price_text = await first_item.locator('[data-aut-id="itemPrice"]').inner_text()


            price = int(''.join(filter(str.isdigit, price_text)))

            await browser.close()
            return {"title": title, "price": price}

        except Exception as e:
            print(f"⚠️ Scraper Error: {e}")

            await page.screenshot(path="last_error.png")
            print("📸 Error screenshot saved as 'last_error.png'. Open it from the Colab file sidebar.")
            await browser.close()
            return None